<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
<span><img src="media/inria-25-alpha.png" /></span>
</div>

# récrire l'historique

In [ ]:
# ce sera toujours notre façon de commencer
[ -f scripts/helpers.sh ] && source scripts/helpers.sh

## `repo-rebase`

In [ ]:
cd $TOP

# pour pouvoir recommencer le scénario depuis le début
# je nettoie complètement ce qu'on a pu faire précédemment
if [ -d repo-rebase ]; then
    echo "on repart d'un directory vide"
    rm -rf repo-rebase
fi

# on le crée
mkdir repo-rebase

# on va dedans
cd repo-rebase

## rappel : un commit est immutable

en informatique, on fait la distinction entre  
objets *mutables* et *immutables*

* un terme savant pour désigner les objets  
  qui peuvent changer

* un ficher par exemple est un objet mutable

* un commit est par contre **immutable**
* dit autrement, une fois qu'il est créé  
  on ne peut plus le modifier  

* mais on peut en **créer un autre**

## attention aux commits poussés

avant d'aller plus loin, souvenez-vous de cette règle d'or

> ne **jamais** bidouiller (recréer) un commit déjà publié (poussé)

si quelqu'un a déjà tiré ce commit, et vous le remplacez  
par un autre, au deuxième `pull` votre collègue  
va déclencher un désordre monstrueux

on en reparlera

## un repo

In [ ]:
$SCRIPTS/do rebase-init
$SCRIPTS/do rebase-master-branch
$SCRIPTS/do rebase-devel-branch

In [ ]:
# rappel:
# git l = git log --oneline --graph
git l --all

## modifier le dernier commit

le cas le plus simple de récriture de l'histoire :  
vous voulez modifier le commit que vous venez de faire  
à cause d'une faute d'orthographe dans le message


```
git commit --amend
```

si vous avez ajouté des changements dans l'index entretemps,  
ils feront partie du nouveau commit 

In [ ]:
git l

In [ ]:
# récrire le dernier commit
git commit --amend --message C

In [ ]:
git l

In [ ]:
# le commit 'OOPS' est 
# toujours là quelque part
# mais on ne le parcourt pas
git l --all

## reconstruire avec `rebase` 

un outil permettant de rejouer une suite de changements  



![](media/merge-vs-rebase.png)

`rebase` *vs* `merge` :

les deux résultats `F` et `E'`  
ont un contenu identique  
mais la topologie est  
évidemment très différente

In [ ]:
# juste pour garder une référence
git branch old-devel devel

In [ ]:
git l --all

In [ ]:
# ça se lit comme ceci
# reconstruire la branche devel 
# au dessus de la branche master
git rebase master devel

In [ ]:
git l --all

## F == E'

pour s'assurer qu'on a bien le même contenu qu'avec un merge

In [ ]:
git checkout -b merging master
git merge old-devel --message F

In [ ]:
git l --all

In [ ]:
# pas de différence entre les deux contenus
git diff merging devel

## `pull --rebase`

souvenez-vous des cas où il faut jouer à `pull-push`  
i.e. les cas où deux personnes partent d'un même commit  
et où on l'un des deux ne peut pas pousser  
il doit d'abord faire `pull` (cf. *fast-forward*) 

dans ces cas-là si on préfère ne pas créer de diamant  
envisagez dans ce cas-là de faire un `pull --rebase`  
c'est souvent une option proposée dans les diverses UI

## `rebase -i`

on signale enfin (détails à creuser par vous-mêmes)  
le **mode interactif** de rebase :

* toujours sur des commits **non publiés**
* on peut récrire une suite de commits pour
* changer leur ordre
* en regrouper
* en enlever
* ...

ce qui est l'outil idéal pour produire un historique propre

## ne pas utiliser `push --force`

revenons au slide "*attention aux commits poussés*"  
si vous avez déjà publié (poussé) un commit  
et que vous le récrivez - par exemple avec `commit --amend`  

* vous ne pouvez pas pousser le nouveau commit,  
  car ce n'est pas un fast-forward (fig. suivante)

* `git push` vous offre une option `--force`  
  qui ignore le problème  

* mais un tiers a déjà tiré le premier commit  
  ça va créer un gros désordre  
  
**Attention** donc à  

* éviter de récrire un commit déjà publié  
* réserver `push --force` à des cas bien contrôlés 



![](media/force-push-is-evil.png)

En partant d'un état stable, avec 2 contributeurs; alice publie le commit B, et se rend compte qu'elle a besoin de l'amender.

En fait, c'est une situation possiblement problématique; si bob a déjà eu le temps de tirer ce commit, il est de loin préférable de s'abstenir de modifier B (et de créer plutôt un commit au dessus de B)

Tout d'abord remarquons que si alice a déjà publié B et qu'elle récrit B', elle ne peut pas pousser B' sur github, qui a ce stade a sa branche master sur B, on n'est donc pas dans le cas d'un fast-forward.

Alice peut decider de passer outre mais pour cela elle doit invoquer `git push --force`. 

Ce serait une mauvaise idée dans ce cas car, sauf à contacter bob directement pour qu'il nettoie son repo, la prochaine fois que bob va tirer il va créer un commit B'', qui sera le troisième commit qui traite du même changement, et avec un fort risque de conflit en plus !

## exercice - amend

1. créez un commit  
   modifiez le message avec `amend`
   
1. même scénario, mais vous créez une branche `bookmark`  
   juste avant le `amend` pour vérifier  
   que le premier commit est toujours présent dans le repo
   
1. créez un commit  
   ajoutez des changements dans l'index  
   faites un `amend` et constatez que l'indez est vide
   
1. utilisez `amend` pour modifier l'auteur du commit  
   voyez la doc de `commit` pour cela

## exercice - rebase 

1. créer un repo avec 5 commits init, bugfix1, feature1, bugfix2, feature2  
1. utiliser `rebase -i` pour récrire au dessus de init  
   le même contenu en seulement 2 commit feature, bugfix
1. créer une branche au niveau de init, lui appliquer le commit bugfix  
   (*hint*: voir la commande `cherry-pick`)